In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
from pandas.api.types import union_categoricals
from itertools import islice
import re
import addfips
import requests
import urllib
import time
DC_STATEHOOD = 1 # Enables DC to be included in the state list
import us
import pickle
import rapidfuzz

Obtain USDA MyMarketNews reports using their API

In [2]:
# USDA API key from my account
# Import API key stored in text file
with open("../tools/mymarketnews_api_key.txt") as f:
    lines = f.readlines()

api_key = lines[0]

In [3]:
# Base URL to call API
base_url = 'https://marsapi.ams.usda.gov/services/v1.2/reports/'

# Use API key as basic authentication username, password not needed
from requests.auth import HTTPBasicAuth
api_auth = HTTPBasicAuth(api_key, '')

In [4]:
# # Make request
# request_url = base_url
# r = requests.get(url = request_url, auth = api_auth)

# # Put response into spreadsheet for analysis
# r_json = r.json()
# data = [x for x in r_json]
# all_reports = pd.DataFrame(data)
# all_reports.to_excel("mymarketnews_reports.xlsx")

In [5]:
all_reports = pd.read_excel("mymarketnews_reports.xlsx")
# Get slug ID and name of all terminal (wholesale) market reports
terminal_markets = all_reports[all_reports['market_types'] == '[\'Terminal\']']

# Remove foreign terminal markets
terminal_markets = terminal_markets[terminal_markets['offices'] != '[\'Washington, DC International - SC\']']

# Get slug ID and name of all shipping point reports
shipping_points = all_reports[all_reports['market_types'] == '[\'Shipping Point\']']

In [6]:
# report_header_df = pd.DataFrame()

# for slug_id in terminal_markets['slug_id']:
#     print(str(slug_id))

#     time.sleep(1) # stagger each request by 1 sec

#     # Pull report headers using Slug ID
#     # Define request
#     request_url = base_url + str(slug_id) # + '?q=' + 'report_begin_date=12/19/2024' + '&allSections=true'

#     # Make request
#     r = requests.get(url = request_url, auth = api_auth)

#     # Put response into dataframe
#     r_json = r.json()
#     response_df = pd.DataFrame(r_json['results'])
#     report_header_df = pd.concat([report_header_df, response_df])

# # Put report headers into big parquet
# report_header_df.to_parquet("../binaries/mymarketnews_terminal_report_headers.parquet", index=False)

In [7]:
# Create directories to store CSVs and binaries
try:
    Path.mkdir(f"../Data/mymarketnews_reports/")
except (FileExistsError):
    print('Data folder already exists.')

for year in range(2004, 2025):
    try:
        Path.mkdir(f"../Data/mymarketnews_reports/mymarketnews_reports_{year}")
    except (FileExistsError) as err:
        print('Year ' + f'{year}' + ' data folder already exists.')

Data folder already exists.
Year 2004 data folder already exists.
Year 2005 data folder already exists.
Year 2006 data folder already exists.
Year 2007 data folder already exists.
Year 2008 data folder already exists.
Year 2009 data folder already exists.
Year 2010 data folder already exists.
Year 2011 data folder already exists.
Year 2012 data folder already exists.
Year 2013 data folder already exists.
Year 2014 data folder already exists.
Year 2015 data folder already exists.
Year 2016 data folder already exists.
Year 2017 data folder already exists.
Year 2018 data folder already exists.
Year 2019 data folder already exists.
Year 2020 data folder already exists.
Year 2021 data folder already exists.
Year 2022 data folder already exists.
Year 2023 data folder already exists.
Year 2024 data folder already exists.


In [8]:
# Pull terminal market report details
for year in range(2004, 2025):
    
    terminal_report_details_df = pd.DataFrame()

    for slug_id in terminal_markets['slug_id']:
        
        # If slug and year is already downloaded, we can skip it
        slug_year_path = Path(f"../Data/mymarketnews_reports/mymarketnews_reports_{year}/mymarketnews_terminal_market_reports_year_{year}_slug_{slug_id}.csv")
        if slug_year_path.is_file():
            print('Slug ID', str(slug_id), 'year', str(year), 'already downloaded.')
            continue

        print('Slug ID is', str(slug_id), 'year is', str(year))

        time.sleep(5) # stagger each request by 5 sec

        # Pull report details using Slug ID
        # Define request
        request_url = base_url + str(slug_id) + '?q=' + f'report_begin_date=01/01/{year}:12/31/{year}' + '&allSections=true'

        # Make request
        r = requests.get(url = request_url, auth = api_auth)

        # Put response into dataframe
        try:
            r_json = r.json()
        except:
            r = requests.get(url = request_url, auth = api_auth)

        r_json = r.json()
        results = r_json[1]['results']
        result_df = pd.DataFrame(results)

        print('Report has ' + str(len(result_df)) + ' entries')

        # terminal_report_details_df = pd.concat([terminal_report_details_df, result_df])
        result_df.to_csv(f"../Data/mymarketnews_reports/mymarketnews_reports_{year}/mymarketnews_terminal_market_reports_year_{year}_slug_{slug_id}.csv", index=False)

Slug ID 2277 year 2004 already downloaded.
Slug ID 2278 year 2004 already downloaded.
Slug ID 2279 year 2004 already downloaded.
Slug ID 2280 year 2004 already downloaded.
Slug ID 2281 year 2004 already downloaded.
Slug ID 2282 year 2004 already downloaded.
Slug ID 2283 year 2004 already downloaded.
Slug ID 2284 year 2004 already downloaded.
Slug ID 2285 year 2004 already downloaded.
Slug ID 2286 year 2004 already downloaded.
Slug ID 2287 year 2004 already downloaded.
Slug ID 2288 year 2004 already downloaded.
Slug ID 2289 year 2004 already downloaded.
Slug ID 2290 year 2004 already downloaded.
Slug ID 2291 year 2004 already downloaded.
Slug ID 2292 year 2004 already downloaded.
Slug ID 2293 year 2004 already downloaded.
Slug ID 2294 year 2004 already downloaded.
Slug ID 2295 year 2004 already downloaded.
Slug ID 2296 year 2004 already downloaded.
Slug ID 2297 year 2004 already downloaded.
Slug ID 2298 year 2004 already downloaded.
Slug ID 2299 year 2004 already downloaded.
Slug ID 230

In [9]:
# Pull shipping point report details
for year in range(2004, 2025):
    shipping_point_report_details_df = pd.DataFrame()
    print('Year is ' + str(year))

    for slug_id in shipping_points['slug_id']:

        # If slug and year is already downloaded, we can skip it
        slug_year_path = Path(f"../Data/mymarketnews_reports/mymarketnews_reports_{year}/mymarketnews_shipping_point_reports_year_{year}_slug_{slug_id}.csv")
        if slug_year_path.is_file():
            print('Slug ID', str(slug_id), 'year', str(year), 'already downloaded.')
            continue

        print('Slug ID is', str(slug_id), 'year is', str(year))

        time.sleep(5) # stagger each request by 5 sec

        # Pull report details using Slug ID
        # Define request
        request_url = base_url + str(slug_id) + '?q=' + f'report_begin_date=01/01/{year}:12/31/{year}' + '&allSections=true'

        # Make request
        r = requests.get(url = request_url, auth = api_auth)

        # Put response into dataframe
        r_json = r.json()
        results = r_json[1]['results']
        result_df = pd.DataFrame(results)

        print('Report has ' + str(len(result_df)) + ' entries')

        # shipping_point_report_details_df = pd.concat([shipping_point_report_details_df, result_df])
        result_df.to_csv(f"../Data/mymarketnews_reports/mymarketnews_reports_{year}/mymarketnews_shipping_point_reports_year_{year}_slug_{slug_id}.csv", index=False)

Year is 2004
Slug ID 2386 year 2004 already downloaded.
Slug ID 2387 year 2004 already downloaded.
Slug ID 2388 year 2004 already downloaded.
Slug ID 2389 year 2004 already downloaded.
Slug ID 2390 year 2004 already downloaded.
Slug ID 2391 year 2004 already downloaded.
Slug ID 2392 year 2004 already downloaded.
Slug ID 2393 year 2004 already downloaded.
Slug ID 2394 year 2004 already downloaded.
Slug ID 2395 year 2004 already downloaded.
Slug ID 2396 year 2004 already downloaded.
Slug ID 2397 year 2004 already downloaded.
Slug ID 2398 year 2004 already downloaded.
Slug ID 2399 year 2004 already downloaded.
Slug ID 2400 year 2004 already downloaded.
Slug ID 2401 year 2004 already downloaded.
Slug ID 2402 year 2004 already downloaded.
Slug ID 2403 year 2004 already downloaded.
Slug ID 2404 year 2004 already downloaded.
Slug ID 2405 year 2004 already downloaded.
Slug ID 2406 year 2004 already downloaded.
Slug ID 2407 year 2004 already downloaded.
Slug ID 2408 year 2004 already downloaded

In [10]:
for year in range(2004, 2025):
    year_df = pd.DataFrame()

    # Skip if binary already created
    binary_year_path = Path(f"../binaries/mymarketnews_shipping_point_reports_{year}.parquet")
    if binary_year_path.is_file():
        print(str(year), 'already converted')
        continue
    else:
        print('Converting ', year)
        for csv_path in Path(f"../Data/mymarketnews_reports/mymarketnews_reports_{year}").iterdir():
            if 'shipping_point' in csv_path.stem:
                try:
                    report_df = pd.read_csv(csv_path, dtype = str, engine = 'c')
                    year_df = pd.concat([year_df, report_df], ignore_index = True)
                except (pd.errors.EmptyDataError):
                    print('Empty file:', csv_path)
    
    # Shipping Point reports use district instead of origin
    year_df.rename(columns = {'district': 'origin'}, inplace = True)
    year_df.to_parquet(f"../binaries/mymarketnews_shipping_point_reports_{year}.parquet")
                

Converting  2004


Empty file: ..\Data\mymarketnews_reports\mymarketnews_reports_2004\mymarketnews_shipping_point_reports_year_2004_slug_3134.csv
Converting  2005
Empty file: ..\Data\mymarketnews_reports\mymarketnews_reports_2005\mymarketnews_shipping_point_reports_year_2005_slug_2392.csv
Empty file: ..\Data\mymarketnews_reports\mymarketnews_reports_2005\mymarketnews_shipping_point_reports_year_2005_slug_2411.csv
Empty file: ..\Data\mymarketnews_reports\mymarketnews_reports_2005\mymarketnews_shipping_point_reports_year_2005_slug_3134.csv
Converting  2006
Empty file: ..\Data\mymarketnews_reports\mymarketnews_reports_2006\mymarketnews_shipping_point_reports_year_2006_slug_2392.csv
Empty file: ..\Data\mymarketnews_reports\mymarketnews_reports_2006\mymarketnews_shipping_point_reports_year_2006_slug_2411.csv
Empty file: ..\Data\mymarketnews_reports\mymarketnews_reports_2006\mymarketnews_shipping_point_reports_year_2006_slug_3134.csv
Converting  2007
Empty file: ..\Data\mymarketnews_reports\mymarketnews_report

In [11]:
for year in range(2004, 2025):
    year_df = pd.DataFrame()

    # Skip if binary already created
    binary_year_path = Path(f"../binaries/mymarketnews_terminal_market_reports_{year}.parquet")
    if binary_year_path.is_file():
        print(str(year), 'already converted')
        continue
    else:
        print('Converting ', year)
        for csv_path in Path(f"../Data/mymarketnews_reports/mymarketnews_reports_{year}").iterdir():
            if 'terminal_market' in csv_path.stem:
                try:
                    report_df = pd.read_csv(csv_path, dtype = str, engine = 'c')
                    year_df = pd.concat([year_df, report_df], ignore_index = True)
                except (pd.errors.EmptyDataError):
                    print('Empty file:', csv_path)
    
    year_df.to_parquet(f"../binaries/mymarketnews_terminal_market_reports_{year}.parquet")
                

2004 already converted
2005 already converted
2006 already converted
2007 already converted
2008 already converted
2009 already converted
2010 already converted
2011 already converted
2012 already converted
2013 already converted
2014 already converted
2015 already converted
2016 already converted
2017 already converted
2018 already converted
2019 already converted
2020 already converted
2021 already converted
2022 already converted
2023 already converted
2024 already converted


Add origin state to our data

Define a function that reads origin string from market reports and determines origin state

In [12]:
def report_origin_state_search(origin_str):
    # Dict of states and abbreviations
    state_abbr_dict = {}
    for state in us.STATES:
        state_name = state.name.upper()
        state_abbr = state.abbr
        state_abbr_dict[state_name] = state_abbr

    # List of states and abbreviation
    state_abbr_list = list(state_abbr_dict.values())
    state_name_list = list(state_abbr_dict.keys())

    if origin_str is None:
        return []

    # Extracts a list of states from a single origin string
    origin_str_upper = origin_str.upper()
    found_states = set()

    # Strategy 1: Search for full state names (longest names first to avoid partial matches)
    # e.g., match "NORTH CAROLINA" before "CAROLINA"
    sorted_state_names = sorted(state_name_list, key=len, reverse=True)
    for state_name in sorted_state_names:
        # Use word boundaries to ensure whole word matching
        if re.search(r'\b' + re.escape(state_name) + r'\b', origin_str_upper):
            found_states.add(state_name)

    # Strategy 2: Search for state abbreviations (e.g., "CA", "AZ", "WV")
    # This is effective for patterns like "(MD, PA, VA, WV)" or "... CA, ... AZ"
    # We look for two consecutive uppercase letters that form a valid abbreviation
    potential_abbrs = re.findall(r'\b([A-Z]{2})\b', origin_str_upper)
    for abbr in potential_abbrs:
        if abbr in state_abbr_list:
            found_states.add(abbr)

    # We leave imports blank
    if re.search(r'\b' + re.escape('CROSSINGS') + r'\b', origin_str_upper):
        return([])
    
    if re.search(r'\b' + re.escape('IMPORTS') + r'\b', origin_str_upper):
        return([])

    # Return list of found states
    list_of_found_states = []
    for found_state in found_states:
        list_of_found_states.append(us.states.lookup(found_state).name)

    return(list_of_found_states)

In [13]:
# Dict of state names with fips
state_fips_dict = {}
for state in us.STATES:
    state_name = state.name
    state_fips = state.fips
    state_fips_dict[state_name] = state_fips

In [14]:
# Add state of origin to Shipping Point reports
for year in range(2004, 2025):
    print('Doing', year)
    df = pd.read_parquet(f"../binaries/mymarketnews_shipping_point_reports_{year}.parquet")
    origins = df['origin'].drop_duplicates(ignore_index = True)

    # Create a dict mapping district names to states
    origin_dict = {}
    for origin in origins:
        origin_dict[origin] = report_origin_state_search(origin)

    # ARKANSAS VALLEY COLORADO refers only to Colorado as Arkansas Valley is a region in Colorado
    origin_dict['ARKANSAS VALLEY COLORADO'] = ['Colorado']

    df['state_of_origin'] = df['origin'].map(origin_dict)

    df.to_parquet(f"../binaries/mymarketnews_shipping_point_reports_{year}.parquet")

Doing 2004
Doing 2005
Doing 2006
Doing 2007
Doing 2008
Doing 2009
Doing 2010
Doing 2011
Doing 2012
Doing 2013
Doing 2014
Doing 2015
Doing 2016
Doing 2017
Doing 2018
Doing 2019
Doing 2020
Doing 2021
Doing 2022
Doing 2023
Doing 2024


In [15]:
for year in range(2004, 2025):
    print('Doing', year)
    df = pd.read_parquet(f"../binaries/mymarketnews_terminal_market_reports_{year}.parquet")
    origins = df['origin'].drop_duplicates(ignore_index = True)

    # Create a dict mapping district names to states
    origin_dict = {}
    for origin in origins:
        origin_dict[origin] = report_origin_state_search(origin)

    # ARKANSAS VALLEY COLORADO refers only to Colorado as Arkansas Valley is a region in Colorado
    origin_dict['ARKANSAS VALLEY COLORADO'] = ['Colorado']

    df['state_of_origin'] = df['origin'].map(origin_dict)

    df.to_parquet(f"../binaries/mymarketnews_terminal_market_reports_{year}.parquet")

Doing 2004
Doing 2005
Doing 2006
Doing 2007
Doing 2008
Doing 2009
Doing 2010
Doing 2011
Doing 2012
Doing 2013
Doing 2014
Doing 2015
Doing 2016
Doing 2017
Doing 2018
Doing 2019
Doing 2020
Doing 2021
Doing 2022
Doing 2023
Doing 2024
